참고 : Generative Deep Learning 미술관에 GAN 딥러닝 실전 프로젝트 - 데이비드 포스터 지음 (출판사 : 한빛미디어)

참고 : https://sunghan-kim.github.io/ml/3min-dl-ch09/

# GAN(Generative Adversarial Network) : 생성적 적대 신경망
- 생성자(generator), 판별자(discriminator) 네트워크 2개가 경쟁하는 것
- 생성자 : 원본 데이터셋에서 샘플링한 것처럼 보이는 샘플로 변환 (진짜같은 가짜 생성)
- 판별자 : 원본 데이터셋에서 추출한 샘플인지 생성자가 만든 가짜인지를 구별 (진짜인지, 가짜인지 구별)
- <img src="https://1.bp.blogspot.com/-n9mKBe3m9zw/WZkBAS_oUcI/AAAAAAAAAIU/WKIHqZp7z_IvQ-arRsEvWDp8C8foPDc2wCLcBGAs/s1600/6.png" width=600>
- 생성자는 더 진짜같은 가짜를 만들어내고, 판별자는 정확하게 진짜와 가짜를 구별하는 능력을 유지하도록 학습한다.

In [16]:
# 구글 드라이브 import
from google.colab import drive
drive.mount("/content/drive")

# 기본 환경 설정
!git clone https://github.com/rickiepark/GDL_code.git
!git pull
!conda create -n generative python=3.6 ipykernel
!pip install virtualenv virtualenvwrapper
!mkvirtualenv generative
!pip install absl-py appnope backcall bleach cloudpickle cycler dask decorator defusedxml entrypoints gast grpcio h5py ipykernel ipython ipython-genutils ipywidgets jedi Jinja2 jsonschema jupyter jupyter-client jupyter-console jupyter-core Keras Keras-Applications Keras-Preprocessing kiwisolver Markdown MarkupSafe matplotlib mistune music21 nbconvert nbformat networkx notebook numpy pandas pandocfilters parso pexpect pickleshare Pillow prometheus-client prompt-toolkit protobuf ptyprocess pydot Pygments pyparsing python-dateutil pytz PyWavelets PyYAML pyzmq qtconsole scikit-image scipy Send2Trash six tensorboard tensorflow termcolor terminado testpath toolz tornado traitlets wcwidth webencodings Werkzeug widgetsnbextension

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
fatal: destination path 'GDL_code' already exists and is not an empty directory.
fatal: not a git repository (or any of the parent directories): .git
/bin/bash: conda: command not found
/bin/bash: mkvirtualenv: command not found


# 애니멀간 데이터
- 28x28 픽셀, 흑백 낙서 이미지 데이터
- 주제별로 labeling 되어있음

# GAN 모델링

In [17]:
# GAN class 정의

from keras.layers import Input, Conv2D, Flatten, Dense, Conv2DTranspose, Reshape, Lambda, Activation, BatchNormalization, LeakyReLU, Dropout, ZeroPadding2D, UpSampling2D
# from keras.layers.merge import _Merge
# keras 2.x 버전에서는 작동 안 함
# 대신에 Add나 concatenate 사용
from keras.layers import Add, Concatenate

from keras.models import Model, Sequential
from keras import backend as K
from keras.optimizers import Adam, RMSprop
from keras.utils import plot_model
from keras.initializers import RandomNormal

import numpy as np
import json
import os
import pickle as pkl
import matplotlib.pyplot as plt

def concatenate(x1, x2):
  merged = Concatenate([x1, x2])
  return merged

class GAN():
  # parameter 설정, model build, compile
    def __init__(self
        , input_dim
        , discriminator_conv_filters
        , discriminator_conv_kernel_size
        , discriminator_conv_strides
        , discriminator_batch_norm_momentum
        , discriminator_activation
        , discriminator_dropout_rate
        , discriminator_learning_rate
        , generator_initial_dense_layer_size
        , generator_upsample
        , generator_conv_filters
        , generator_conv_kernel_size
        , generator_conv_strides
        , generator_batch_norm_momentum
        , generator_activation
        , generator_dropout_rate
        , generator_learning_rate
        , optimiser
        , z_dim
        ):

        self.name = 'gan'

        # 파라미터 초기화
        self.input_dim = input_dim
        self.discriminator_conv_filters = discriminator_conv_filters
        self.discriminator_conv_kernel_size = discriminator_conv_kernel_size
        self.discriminator_conv_strides = discriminator_conv_strides
        self.discriminator_batch_norm_momentum = discriminator_batch_norm_momentum
        self.discriminator_activation = discriminator_activation
        self.discriminator_dropout_rate = discriminator_dropout_rate
        self.discriminator_learning_rate = discriminator_learning_rate

        self.generator_initial_dense_layer_size = generator_initial_dense_layer_size
        self.generator_upsample = generator_upsample
        self.generator_conv_filters = generator_conv_filters
        self.generator_conv_kernel_size = generator_conv_kernel_size
        self.generator_conv_strides = generator_conv_strides
        self.generator_batch_norm_momentum = generator_batch_norm_momentum
        self.generator_activation = generator_activation
        self.generator_dropout_rate = generator_dropout_rate
        self.generator_learning_rate = generator_learning_rate
        
        self.optimiser = optimiser
        self.z_dim = z_dim

        # discriminator, generator의 layer 갯수
        self.n_layers_discriminator = len(discriminator_conv_filters)
        self.n_layers_generator = len(generator_conv_filters)

        # 정규분포로 초깃값 설정
        self.weight_init = RandomNormal(mean=0., stddev=0.02)

        self.d_losses = []
        self.g_losses = []

        self.epoch = 0

        # discriminator, generator 모델 생성 메소드
        self._build_discriminator()
        self._build_generator()

        self._build_adversarial()

    # 활성화 함수 지정
    def get_activation(self, activation):
        if activation == 'leaky_relu':
            layer = LeakyReLU(alpha = 0.2)
        else:
            layer = Activation(activation)
        return layer

    # discriminator(판별자) 생성
    # Conv2D, BatchNormalization, activation, Dropout, Flatten, Dense, Model
    # ------------------------------------------------------------------------
    # input : image
    # output : 진짜 이미지일 확률. 0~1 사이의 값.
    # ------------------------------------------------------------------------
    def _build_discriminator(self):

        ### THE discriminator
        discriminator_input = Input(shape=self.input_dim, name='discriminator_input')  # discriminator의 input 정의

        x = discriminator_input

        # 합성곱 층(convolution layer) 쌓기
        for i in range(self.n_layers_discriminator):

            x = Conv2D(
                filters = self.discriminator_conv_filters[i]
                , kernel_size = self.discriminator_conv_kernel_size[i]
                , strides = self.discriminator_conv_strides[i]
                , padding = 'same'
                , name = 'discriminator_conv_' + str(i)
                , kernel_initializer = self.weight_init
                )(x)

            if self.discriminator_batch_norm_momentum and i > 0:
                x = BatchNormalization(momentum = self.discriminator_batch_norm_momentum)(x)

            x = self.get_activation(self.discriminator_activation)(x)

            if self.discriminator_dropout_rate:
                x = Dropout(rate = self.discriminator_dropout_rate)(x)

        # 마지막 합성곱 층 -> Flatten -> 평탄화 작업을 거쳐서 벡터화.
        # 이유? Dense 층에 들어가야 하므로.
        x = Flatten()(x)
        
        # Dense 층의 유닛의 갯수 : 1개
        # 이유? input인 이미지가 얼마나 진짜 이미지에 가까운지 확률값 1개만 출력하기 때문.

        # 활성화함수 : sigmoid 함수
        # 이유? 0~1 사이의 값을 출력해주기 때문.
        discriminator_output = Dense(1, activation='sigmoid', kernel_initializer = self.weight_init)(x)

        # keras의 Model : input layer, output layer를 받음.
        self.discriminator = Model(discriminator_input, discriminator_output)


    # generator(생성자) 생성
    # Dense, BatchNormalization, activation, Reshape, Dropout, UpSampling2D, Conv2D, Model
    # ------------------------------------------------------------------------
    # input : noise
    # output : image
    # ------------------------------------------------------------------------
    def _build_generator(self):

        ### THE generator

        # 입력받은 길이의 노이즈 벡터 생성
        generator_input = Input(shape=(self.z_dim,), name='generator_input')

        x = generator_input

        x = Dense(np.prod(self.generator_initial_dense_layer_size), kernel_initializer = self.weight_init)(x)

        if self.generator_batch_norm_momentum:
            x = BatchNormalization(momentum = self.generator_batch_norm_momentum)(x)

        x = self.get_activation(self.generator_activation)(x)

        x = Reshape(self.generator_initial_dense_layer_size)(x)

        if self.generator_dropout_rate:
            x = Dropout(rate = self.generator_dropout_rate)(x)

        for i in range(self.n_layers_generator):

            if self.generator_upsample[i] == 2:
                x = UpSampling2D()(x)
                x = Conv2D(
                    filters = self.generator_conv_filters[i]
                    , kernel_size = self.generator_conv_kernel_size[i]
                    , padding = 'same'
                    , name = 'generator_conv_' + str(i)
                    , kernel_initializer = self.weight_init
                )(x)
            else:

                x = Conv2DTranspose(
                    filters = self.generator_conv_filters[i]
                    , kernel_size = self.generator_conv_kernel_size[i]
                    , padding = 'same'
                    , strides = self.generator_conv_strides[i]
                    , name = 'generator_conv_' + str(i)
                    , kernel_initializer = self.weight_init
                    )(x)

            if i < self.n_layers_generator - 1:

                if self.generator_batch_norm_momentum:
                    x = BatchNormalization(momentum = self.generator_batch_norm_momentum)(x)

                x = self.get_activation(self.generator_activation)(x)
                    
                
            else:
                
                # 마지막 Conv2D 층 다음 활성화 함수 tanh 사용
                # 이유 ? 출력을 원본 이미지와 같은 -1 ~ 1 범위로 변환하기 위해서.
                x = Activation('tanh')(x)


        generator_output = x

        self.generator = Model(generator_input, generator_output)

       
    # optimiser 지정
    def get_opti(self, lr):
        if self.optimiser == 'adam':
            opti = Adam(lr=lr, beta_1=0.5)
        elif self.optimiser == 'rmsprop':
            opti = RMSprop(lr=lr)
        else:
            opti = Adam(lr=lr)

        return opti


    def set_trainable(self, m, val):
        m.trainable = val
        for l in m.layers:
            l.trainable = val

    # GAN 모델링
    def _build_adversarial(self):
        
        ### COMPILE DISCRIMINATOR

        # target : 0, 1 -> 2개
        # 활성화 함수 : sigmoid
        # 출력층의 유닛의 갯수 : 1개

        # -> binary_crossentropy 사용
        self.discriminator.compile(
        optimizer=self.get_opti(self.discriminator_learning_rate)  
        , loss = 'binary_crossentropy'
        ,  metrics = ['accuracy']
        )
        
        ### COMPILE THE FULL GAN
        
        # 생성자를 훈련하기 위한 모델 컴파일
        # 판별자의 가중치 동결 ; 컴파일한 판별자 모델이 영향을 받지 않도록 한다.
        self.set_trainable(self.discriminator, False)

        # 잠재 공간 벡터 -> 생성자 -> 판별자 -> 확률 출력하는 모델 생성
        model_input = Input(shape=(self.z_dim,), name='model_input')
        model_output = self.discriminator(self.generator(model_input))
        self.model = Model(model_input, model_output)

        # binary cross entropy를 이용하여 전체 모델을 compile.
        self.model.compile(optimizer=self.get_opti(self.generator_learning_rate) , loss='binary_crossentropy', metrics=['accuracy'])

        self.set_trainable(self.discriminator, True)



    # discriminator(판별자) 훈련
    # 판별자, 생성자 순서대로 훈련시킨다.
    def train_discriminator(self, x_train, batch_size, using_generator):

        # 진짜 이미지의 target : 1
        valid = np.ones((batch_size,1))
        # 가짜 이미지의 target : 0
        fake = np.zeros((batch_size,1))

        # 진짜 이미지로 훈련
        if using_generator:
            true_imgs = next(x_train)[0]  # iterator, next 사용
            if true_imgs.shape[0] != batch_size:
                true_imgs = next(x_train)[0]
        else:
            idx = np.random.randint(0, x_train.shape[0], batch_size)
            true_imgs = x_train[idx]
        
        # 가짜 이미지(생성된 이미지)로 훈련
        noise = np.random.normal(0, 1, (batch_size, self.z_dim))
        gen_imgs = self.generator.predict(noise)

        d_loss_real, d_acc_real =   self.discriminator.train_on_batch(true_imgs, valid)
        d_loss_fake, d_acc_fake =   self.discriminator.train_on_batch(gen_imgs, fake)
        d_loss =  0.5 * (d_loss_real + d_loss_fake)
        d_acc = 0.5 * (d_acc_real + d_acc_fake)

        return [d_loss, d_loss_real, d_loss_fake, d_acc, d_acc_real, d_acc_fake]

    # generator(생성자) 훈련
    # 판별자 훈련시켰으니 이제 생성자를 훈련시킨다.

    # 판별자의 가중치 : 동결되었으므로 변하지 X.
    # 생성자의 가중치 : 판별자가 1에 가까운 값으로 예측할 수 있는 이미지를 생성하는 방향으로 업데이트.
    def train_generator(self, batch_size):
        valid = np.ones((batch_size,1))
        noise = np.random.normal(0, 1, (batch_size, self.z_dim))
        return self.model.train_on_batch(noise, valid)


    def train(self, x_train, batch_size, epochs, run_folder
    , print_every_n_batches = 50
    , using_generator = False):

        for epoch in range(self.epoch, self.epoch + epochs):

            d = self.train_discriminator(x_train, batch_size, using_generator)
            g = self.train_generator(batch_size)

            print ("%d [D loss: (%.3f)(R %.3f, F %.3f)] [D acc: (%.3f)(%.3f, %.3f)] [G loss: %.3f] [G acc: %.3f]" % (epoch, d[0], d[1], d[2], d[3], d[4], d[5], g[0], g[1]))

            self.d_losses.append(d)
            self.g_losses.append(g)

            if epoch % print_every_n_batches == 0:
                self.sample_images(run_folder)
                self.model.save_weights(os.path.join(run_folder, 'weights/weights-%d.h5' % (epoch)))
                self.model.save_weights(os.path.join(run_folder, 'weights/weights.h5'))
                self.save_model(run_folder)

            self.epoch += 1

    
    def sample_images(self, run_folder):
        r, c = 5, 5
        noise = np.random.normal(0, 1, (r * c, self.z_dim))
        gen_imgs = self.generator.predict(noise)

        gen_imgs = 0.5 * (gen_imgs + 1)
        gen_imgs = np.clip(gen_imgs, 0, 1)

        fig, axs = plt.subplots(r, c, figsize=(15,15))
        cnt = 0

        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(np.squeeze(gen_imgs[cnt, :,:,:]), cmap = 'gray')
                axs[i,j].axis('off')
                cnt += 1
        fig.savefig(os.path.join(run_folder, "images/sample_%d.png" % self.epoch))
        plt.close()




    
    def plot_model(self, run_folder):
        plot_model(self.model, to_file=os.path.join(run_folder ,'viz/model.png'), show_shapes = True, show_layer_names = True)
        plot_model(self.discriminator, to_file=os.path.join(run_folder ,'viz/discriminator.png'), show_shapes = True, show_layer_names = True)
        plot_model(self.generator, to_file=os.path.join(run_folder ,'viz/generator.png'), show_shapes = True, show_layer_names = True)



    def save(self, folder):

        with open(os.path.join(folder, 'params.pkl'), 'wb') as f:
            pkl.dump([
                self.input_dim
                , self.discriminator_conv_filters
                , self.discriminator_conv_kernel_size
                , self.discriminator_conv_strides
                , self.discriminator_batch_norm_momentum
                , self.discriminator_activation
                , self.discriminator_dropout_rate
                , self.discriminator_learning_rate
                , self.generator_initial_dense_layer_size
                , self.generator_upsample
                , self.generator_conv_filters
                , self.generator_conv_kernel_size
                , self.generator_conv_strides
                , self.generator_batch_norm_momentum
                , self.generator_activation
                , self.generator_dropout_rate
                , self.generator_learning_rate
                , self.optimiser
                , self.z_dim
                ], f)

        self.plot_model(folder)

    def save_model(self, run_folder):
        self.model.save(os.path.join(run_folder, 'model.h5'))
        self.discriminator.save(os.path.join(run_folder, 'discriminator.h5'))
        self.generator.save(os.path.join(run_folder, 'generator.h5'))
        pkl.dump(self, open( os.path.join(run_folder, "obj.pkl"), "wb" ))

    def load_weights(self, filepath):
        self.model.load_weights(filepath)

In [18]:
# gan 정의
gan = GAN(input_dim=(28, 28, 1),
          discriminator_conv_filters=[64, 64, 128, 128],
          discriminator_conv_kernel_size=[5, 5, 5, 5],
          discriminator_conv_strides=[2, 2, 2, 1],
          discriminator_batch_norm_momentum=None,
          discriminator_activation="relu",
          discriminator_dropout_rate=0.4,
          discriminator_learning_rate=0.0008,
          generator_initial_dense_layer_size=(7, 7, 64),
          generator_upsample=[2, 2, 1, 1],
          generator_conv_filters=[128, 64, 64, 1],
          generator_conv_kernel_size=[5, 5, 5, 5],
          generator_conv_strides=[1, 1, 1, 1],
          generator_batch_norm_momentum=0.9,
          generator_activation="relu",
          generator_dropout_rate=None,
          generator_learning_rate=0.0004,
          optimiser="rmsprop",
          z_dim=100)

# 판별자(discriminator)
- input : image
- output : 진짜 이미지일 확률. 0~1 사이의 값.
- 목표 : 이미지가 진짜인지/가짜인지 예측.
- 지도 학습, 이미지 분류 문제
- 합성곱 층 포함되어있음(convolution layer)
- 배치 정규화 많이 사용

# 생성자(generator)
- input : 다변수 표준 정규분포에서 추출한 벡터. 정해진 차원의 잠재 공간 벡터.
- output : 원본 훈련 데이터의 이미지와 동일한 크기의 이미지
- 잠재 공간의 벡터를 조작하여 원본 차원에 있는 이미지의 고수준 특성을 바꿀 수 있는 점에서 VAE(변이형 오토인코더)의 디코더와 비슷한 특징을 가지고 있다.

# 업샘플링(upsampling) 층
- keras의 UpSampling2D 층을 사용하여 tensor의 높이, 너비를 2배로 늘린다.
- 단순히 입력의 각 행과 열을 반복하여 크기를 두 배로 만든다.
- 픽셀 사이 공간을 0으로 채우는 것X, 기존 픽셀값을 사용해 업샘플링
- Conv2DTranspose와 UpSampling2D 방식 모두를 시도해보고 어떤 것이 가장 잘 맞는 지 확인할 것.

# GAN 훈련
- 훈련 데이터셋에서 진짜 샘플을 랜덤하게 선택 & 생성자의 출력을 합쳐서 훈련 세트를 만들어서 판별자를 훈련시킨다.
- label : 진짜(1), 가짜(0)
- 진짜 이미지에 대해서는 1에 가까운 값을 출력, 가짜 이미지에 대해서는 0에 가까운 값을 출력하도록 훈련
- 진짜 이미지가 잠재 공간의 어떤 포인트에 매핑되는 지 알려주는 훈련 세트가 X -> 생성자 훈련 어렵다. -> 판별자의 출력값이 1에 가까워지도록 훈련시켜야 한다.
- 전체 모델을 훈련할 때 생성자의 가중치만 업데이트되도록, 판별자의 가중치를 동결하는 것이 중요하다.
- 판별자의 가중치를 동결하지 X ? -> 생성된 이미지를 진짜라고 여기도록 조종됨.
- 학습률 : GAN에서 주의 깊게 튜닝해야 하는 파라미터!

# 손실 함수(loss function)
- binary cross entropy 사용

GAN을 활용하면 고수준 특성을 스스로 만들 수 있다.

고수준 특성을 구성하기 위해 필수적인 픽셀 사이의 상호 의존 관계를 모델링할 수 있다.